
<div style="width: 100%; text-align: center; padding: 20px;">
  <img align="middle" src="https://eu-images.contentstack.com/v3/assets/blt6b0f74e5591baa03/blt298706ad7dc6401a/6622200b5b7e30e5ebbea827/News_Image_(19).jpg?width=850&auto=webp&quality=95&format=jpg&disable=upscale" alt="Llama 3 demonstration" width="700px">
</div>


<a id = "testing"></a>
# Using LLAMA-3 8b-chat-hf for  Generation: Examples and Implementation

https://huggingface.co/meta-llama/Meta-Llama-3-8B

In [2]:
import os
os.environ['HF_TOKEN'] ="your_hugging_face_token_here"

In [14]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Run the installation commands
if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    !pip install bitsandbytes accelerate
    !pip install gradio
else:
    print("Not running in Google Colab")
    !pip install transformers accelerate datasets bitsandbytes
    !pip install gradio

# Clean up the notebook
clean_notebook()



Notebook cleaned.


**Quantisation Configuration**

In [4]:

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name,device_map="auto",quantization_config=bnb_config)

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [11]:


pipeline =transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    tokenizer=tokenizer,
    max_length=256,
)


def generate_answer(question, temperature):
    try:
        prompt = f'''
        You are an AI content writer.
        Generate a well structured 256 words blog for the given topics below:
        Topics: {question}
        Answer:
        '''
        sequences = pipeline(
            prompt,
            do_sample=True,
            top_k=10,
            temperature=temperature,
            num_return_sequences=1,
            eos_token_id=pipeline.tokenizer.eos_token_id,

        )
        answer = sequences[0]['generated_text'][len(prompt):].strip()

        return answer

    except Exception as e:
        print("Error generating answer:", e)
        return None

In [7]:
question = "วิธีทำข้าวมันไก่"

answer = generate_answer(question, temperature= 0.9)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Generated answer: วิธีทำข้าวมันไก่
        Step 1: กดปุ่ม Start เพื่อเปิดเครื่องทำขาไก่
        Step 2: นำไก่ลงในกระทะ
        Step 3: กดปุ่ม On เพื่อให้ไก่ทำขา
        Step 4: เมื่อไก่ทำขาได้แล้ว ให้กดปุ่ม Off เพื่อให้เครื่องทำขาไก่หยุดทำงาน
        Step 5: นำขาไก่ที่ทำขาได้แล้ว ไปใส่ในกระทะ
        Step 6: กดปุ่ม Power เพื่อให้ไก่ทำขา
        Step 7: เมื่อไก่ทำขาได้แล้ว ให้กดปุ่ม Off เพื่อให้เครื่องทำขาไก่หยุดทำงาน
        Step 8:


In [10]:
question = "plastic pollution in our oceans"

answer = generate_answer( question, temperature= 0.85)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Generated answer: - The ocean is one of the most important resources on Earth. It provides food, water, and transportation. However, plastic pollution has become a major problem in recent years. Plastic waste has been found in every ocean on Earth, and it's estimated that there are more than 100 million tons of plastic in the ocean. This waste is damaging the environment and harming wildlife. It's time to take action and reduce plastic waste.
        - One way to reduce plastic waste is to recycle more. By recycling plastic, we can prevent it from ending up in the ocean. Recycling plastic takes energy, but it's worth it to protect the environment.
        - Another way to reduce plastic waste is to use less plastic in the first place. We can buy products that are made from recycled materials, or we can use less plastic when we shop. Reducing plastic waste is an important step towards protecting the environment. 
        - By recycling more and using less plastic, we can help reduce pla

In [12]:
question = "importance of technology in education."
answer = generate_answer(question, temperature= 0.7)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Generated answer: "Technology has become an integral part of our lives. It has transformed the way we live, work, and learn. In today's fast-paced world, technology has become an essential tool for success in education. Here are some reasons why technology is important in education:
        1. Improved Learning Outcomes: Technology can enhance learning outcomes by making learning more interactive, engaging, and personalized. It can also help students learn at their own pace and in a way that suits their learning style.
        2. Access to Information: Technology has made it easier for students to access information and resources online. This can help them become more informed and well-rounded individuals.
        3. Collaboration and Communication: Technology has made it easier for students to collaborate and communicate with their peers and teachers. This can help them build important skills such as teamwork, problem-solving, and critical thinking.
        4. Increased Efficiency: Te

# Run Gradio

In [22]:
import gradio as gr

iface = gr.Interface(
    fn=generate_answer,
    inputs=[
        gr.Textbox(label="Question", placeholder="Enter your question here"),
        gr.Slider(minimum=0.25, maximum=1.0, step=0.1, value=0.85, label="Temperature"),
    ],
    outputs="text",
    title="AI Content Writer",
    description="Generate a well-structured 200-word blog post for the given topic.",
    examples=[
        ["How to change the world"],
        ["วิธีทำข้าวมันไก่ให้อร่อย"],['How to Make Delicious Hainanese Chicken Rice'],
        ["The future of artificial intelligence"],
    ],
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b5a31d0fdcd844c9e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<a id = "groq"></a>
# Rapid Inference with LLAMA-3 70b on Groq Playground

In [ ]:
!pip install -q groq

In [ ]:
import os
import time
from groq import Groq

client = Groq(
    api_key='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'
)

In [ ]:
def generate_answer_groq(question):
    prompt = f"""
    You are an AI content writer.
    Generate a well-structured 100-word blog for the given topics below:
    Topics: {question}
    Answer:
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192",
    )

    return chat_completion.choices[0].message.content

In [ ]:
start_time = time.time()
print(generate_answer_groq("What were the key causes of the Industrial Revolution?"))
end_time = time.time()
print(f"Time Taken: {end_time-start_time} sec")

Here is a 100-word blog on the key causes of the Industrial Revolution:

**The Spark that Ignited the Industrial Revolution: Key Causes Explained**

The Industrial Revolution, a transformative period in human history, was sparked by a combination of factors. **Innovations in Technology** played a significant role, as inventions like the steam engine and power looms enabled mass production. **Agricultural advances**, which increased food production, liberated labor for industrial pursuits. **Natural Resources**, such as coal and iron, fueled industrial growth. Additionally, **British Colonial Empire**'s global trade networks facilitated the exchange of goods and ideas. Lastly, **Capitalist entrepreneurial spirit** drove investment and innovation, culminating in the Industrial Revolution. These interconnected causes laid the foundation for the modern industrial society we know today.
Time Taken: 0.9124588966369629 sec


In [ ]:
start_time = time.time()
print(generate_answer_groq("How you overcame an obstacle or started a new habit"))
end_time = time.time()
print(f"Time Taken: {end_time-start_time} sec")

**From Procrastination to Productivity: My Journey to Creating a Consistent Writing Habit**

As a writer, I knew I had to overcome procrastination to produce high-quality content consistently. My biggest obstacle was starting new projects, only to abandon them halfway through. To break this cycle, I started small. I committed to writing for just 10 minutes each morning, gradually increasing the duration over time. I also created a conducive writing environment and set specific goals for each session. With persistence and self-discipline, I formed a consistent writing habit. Today, I can proudly say that I've written over 50,000 words in the past month alone. This newfound habit has not only boosted my productivity but also instilled a sense of accomplishment and pride in my work.
Time Taken: 1.0379974842071533 sec


<a id = "Python_Programming_Solutions"></a>
# Using LLAMA-3 LLAMA-3-70b for AI-Generated Python Programming Solutions on Groq Playground


In [ ]:
def generate_answer_groq_programming_bot(question):
    prompt = f"""
    You are an AI programming expert.
    Generate a concise answer to the Python programming-related question below:
    Question: {question}
    Answer:
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192",
    )

    return chat_completion.choices[0].message.content

In [ ]:
start_time = time.time()
print(generate_answer_groq_programming_bot("What is the Pythonic way to check if a key exists in a dictionary?"))
end_time = time.time()
print(f"Time Taken: {end_time-start_time} sec")

A classic Python question!

The Pythonic way to check if a key exists in a dictionary is to use the `in` keyword. Here's an example:
```
my_dict = {'a': 1, 'b': 2}

if 'a' in my_dict:
    print("Key 'a' exists in the dictionary")
else:
    print("Key 'a' does not exist in the dictionary")
```
Alternatively, you can use the `dict.get()` method, which returns `None` if the key is not present:
```
if my_dict.get('a') is not None:
    print("Key 'a' exists in the dictionary")
else:
    print("Key 'a' does not exist in the dictionary")
```
However, the `in` keyword is generally preferred, as it is more concise and idiomatic Python.
Time Taken: 0.8880980014801025 sec


In [ ]:
start_time = time.time()
print(generate_answer_groq_programming_bot("write a Python Program to Print the Fibonacci sequence"))
end_time = time.time()
print(f"Time Taken: {end_time-start_time} sec")

Here is a concise Python program to print the Fibonacci sequence:
```
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        print(a, end=" ")
        a, b = b, a + b
    print()

n = int(input("Enter the number of terms: "))
fibonacci(n)
```
This program uses a simple iterative approach to generate the Fibonacci sequence up to the nth term, where `n` is input by the user. The `fibonacci` function uses a loop to calculate each term as the sum of the previous two terms, printing each term as it goes.

Example output:
```
Enter the number of terms: 10
0 1 1 2 3 5 8 13 21 34
```
Note: You can also use recursion to implement the Fibonacci sequence, but the iterative approach is generally more efficient for large values of `n`.
Time Taken: 2.2346713542938232 sec


# Will update Soon